In [32]:
import pandas as pd
import pyodbc 
import numpy as np
import datetime

In [2]:
#initial cleaning function
def clean_sheet(dataframe):
    adj=dataframe.dropna(axis=1,how='all')
    adj=dataframe.set_index(adj.columns[0]) #use stock tickers as the index
    return adj

In [3]:
cc=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result data\{0} 2007-2017.xlsx'.format('Asia'),sheet_name='Index'))

In [66]:
def fundamental_df_clean(region,year,tabs,volume_low,size_low):
    data_dict={}

    for s in tabs:
        df=clean_sheet(pd.read_excel(r'C:\Users\Eric.Li\Documents\Post result data\{0} {1}.xlsx'.\
                                     format(region,year),sheet_name=s,header=1))
        data_dict[s]=df

    fundamental_df=data_dict["Date"].unstack(-1).to_frame()
    fundamental_df.index=fundamental_df.apply(lambda x: x.name[1],axis=1)
    fundamental_df.columns=["Date"]
    
    
    for i in tabs:
        if (i=='Quarter' and data_dict["Quarter"].shape[1]!=data_dict["Date"].shape[1]) or i=='Orig date':
            fundamental_df[i]=pd.concat([data_dict[i],data_dict["Date"].iloc[:,data_dict[i].shape[1]-\
                                                                   data_dict["Date"].shape[1]:]],axis=1).unstack(-1).values
        else:
            try:
                fundamental_df[i]=data_dict[i].unstack(-1).values
            except ValueError:
                pass
        
    fundamental_df=fundamental_df.dropna(how="all",axis=0)
    fundamental_df=fundamental_df[-np.isnat(fundamental_df.Date)]

    fundamental_df["Supersector"]=fundamental_df.apply(lambda x:data_dict["Property"].loc[x.name,"Supersector"]\
                                           if type(x.name)==str else None,axis=1)

    fundamental_df["Sector"]=fundamental_df.apply(lambda x:data_dict["Property"].loc[x.name,"Sector"]\
                                           if type(x.name)==str else None,axis=1)




    if 'Index' in data_dict.keys():
        fundamental_df["Index"]=fundamental_df.apply(lambda x: data_dict["Index"].loc[x.name].iloc[0] if type(x.name)==str\
                                                     else None,axis=1)

    fundamental_df=fundamental_df[fundamental_df.apply(lambda x: type(x.name)==str,axis=1)]
    #Select the right universe
    fundamental_df=fundamental_df[(fundamental_df["Market cap"]>size_low)&(fundamental_df["Volume"]>volume_low)]
    
    fundamental_df.index.name='Ticker'
    
    if "Orig date" in fundamental_df.columns:
        fundamental_df["Orig date"]=fundamental_df.apply(lambda x:pd.Timestamp(x["Orig date"]),axis=1)
        
    return fundamental_df

In [67]:
US_2019=fundamental_df_clean('US','2019',['Date',"Orig date",'Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [14]:
US_2018=fundamental_df_clean('US','2018',['Date',"Orig date",'Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [15]:
US_history=fundamental_df_clean('US','2007-2017',['Date',"Orig date",'Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [59]:
US=US_history.append(US_2018).append(US_2019)

In [5]:
Europe_2018=fundamental_df_clean('Europe','2018',['Date',"Orig date",'Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [6]:
Europe_2019=fundamental_df_clean('Europe','2019',['Date',"Orig date",'Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [7]:
Europe_history=fundamental_df_clean('Europe','2007-2017',['Date',"Orig date",'Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [58]:
Europe=Europe_history.append(Europe_2018).append(Europe_2019)

In [9]:
Asia_2018=fundamental_df_clean('Asia','2018',['Date',"Orig date",'Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [10]:
Asia_2019=fundamental_df_clean('Asia','2019',['Date',"Orig date",'Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [ ]:
Asia_history=fundamental_df_clean('Asia','2007-2017',['Date',"Orig date",'Market cap','Volume','Property',\
                                                          'Quarter'],3,500)

In [57]:
Asia=Asia_history.append(Asia_2018).append(Asia_2019)

In [71]:
US.to_csv(r'C:\Users\Eric.Li\OneDrive\Post result data cache\US_raw_fundamental_df.csv')

In [72]:
Europe.to_csv(r'C:\Users\Eric.Li\OneDrive\Post result data cache\Europe_raw_fundamental_df.csv')

In [73]:
Asia.to_csv(r'C:\Users\Eric.Li\OneDrive\Post result data cache\Asia_raw_fundamental_df.csv')